In [1]:
%pip install -U torch transformers peft trl bitsandbytes tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")

from huggingface_hub import login
login(token = secret_value_0)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
# 0. imports
import torch
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM

from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=3, # smaller lora dimension? original 16
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)


# 1. load a pretrained model
model_name = "mistralai/Mathstral-7B-v0.1"
#model_name = "gpt2"

compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)


#model = AutoModelForCausalLM.from_pretrained(model_name, 
#                                                          quantization_config=bnb_config, 
#)
tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token = tokenizer.unk_token

#model = get_peft_model(model, lora_config)


2024-09-02 15:35:04.279717: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-02 15:35:04.279823: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-02 15:35:04.413567: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/138k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/588k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## dataset loading

In [4]:
%pip install git+https://github.com/julianjandeleit/swarm_descriptions.git@b824af4d1e0a97e9029d309687316acecc8000e6

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Cloning https://github.com/julianjandeleit/swarm_descriptions.git (to revision b824af4d1e0a97e9029d309687316acecc8000e6) to /tmp/pip-req-build-44y6q5qn
  Running command git clone --filter=blob:none --quiet https://github.com/julianjandeleit/swarm_descriptions.git /tmp/pip-req-build-44y6q5qn
  Running command git rev-parse -q --verify 'sha^b824af4d1e0a97e9029d309687316acecc8000e6'
  Running command git fetch -q https://github.com/julianjandeleit/swarm_descriptions.git b824af4d1e0a97e9029d309687316acecc8000e6
  Running command git checkout -q b824af4d1e0a97e9029d309687316acecc8000e6
  Resolved https://github.com/julianjandeleit/swarm_descriptions.git to commit b824af4d1e0a97e9029d309687316acecc8000e6
  Installing build dependencies ... - \ | / - \ | done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for swarm_descriptions: filename=swarm_descriptions-0.1.0-py3-none-any.whl size=28676 sha256=fa7fe9e2ca0f73c25

In [5]:
import pandas as pd
from swarm_descriptions.mission_elements import get_generators, MissionParams
from swarm_descriptions.configfiles import config_to_string
from swarm_descriptions.utils import truncate_floats
import random
import pyarrow as pa
import pyarrow.dataset as ds

generators = get_generators()
MissionParams.sample(*generators)

n_rows = 2500
rows = []
for n in range(n_rows):
    mission = MissionParams.sample(*generators)
    conf = config_to_string(mission.configure())
    conf = truncate_floats(conf)
    desc = random.sample(mission.describe(),1)[0]  
    desc = truncate_floats(desc)
    rows.append({"description": desc, "configuration": conf, "parameters": mission})


dataset = pd.DataFrame(rows)
dataset.head()

,description,configuration,parameters
0,The arena has a radius of 4.55 m. There are th...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...
1,"The circular arena, constructed with 5 walls, ...","<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...
2,The environment consists of a circular arena w...,"<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...
3,"The circular arena, having a radius of 2.44 me...","<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...
4,"In this setting, a circular arena with a radiu...","<?xml version=""1.00"" ?>\n<config>\n <swarm-el...",MissionParams(arena_params=CircularArena(radiu...


In [6]:
print(dataset.iloc[10].description)
print("---")
print(dataset.iloc[10].configuration)
print("---")
print(dataset.iloc[10].parameters)

With a radius of 4.41 meters, the circular arena is made up of 15 walls. In the arena, 4 lights are evenly spread out with intensities 7.65, 6.48, 3.01, 2.24. Uniformly distributed are 24 robots within a radius of 3.05 meters. The primary objective for the swarm is to cover an area of 5.99 by 3.53 meters while maintaining connectivity. 
---
<?xml version="1.00" ?>
<config>
  <swarm-elems>
    <entity quantity="24" max_trials="100">
      <e-puck id="epuck">
        <controller config="automode_bt"/>
      </e-puck>
    </entity>
  </swarm-elems>
  <env-elems>
    <light id="light_0" position="0.91,-0.81,0.00" orientation="360,0,0" color="yellow" intensity="7.65" medium="leds"/>
    <light id="light_1" position="0.33,2.49,0.00" orientation="360,0,0" color="yellow" intensity="6.48" medium="leds"/>
    <light id="light_2" position="0.76,2.58,0.00" orientation="360,0,0" color="yellow" intensity="3.01" medium="leds"/>
    <light id="light_3" position="-2.07,-1.94,0.00" orientation="360,0,0"

In [7]:
import re

def encode_number(text):
    def encode_match(match):
        number = match.group(0)
        integer_part = match.group(1)
        int_len = len(integer_part)
        decimal_part = match.group(3)
        dec_len = len(decimal_part) if decimal_part else 0
        
        if decimal_part:
            prefix = f"<sn>{int_len}.{dec_len}<mn>"
        else:
            prefix = f"<sn>{int_len}<mn>"
        
        return f"{prefix}{number}<en>"

    pattern = r'(\d+)(\.(\d+))?'
    return re.sub(pattern, encode_match, text)


def decode_number(text):
    text = re.sub(r'<sn>[\d\.]+<mn>', '', text)
    text = re.sub(r'<en>', '', text)
    return text


#special_tokens_dict = {'additional_special_tokens': ['<sn>', '<mn>', '<en>']}
#tokenizer.add_special_tokens(special_tokens_dict)

#tokenizer.save_pretrained('path/to/save/tokenizer')

In [8]:
def generate_prompt(sample, tokenizer):
  messages = [
      {"role": "user", "content": sample["description"]+"\nGenerate the xml configuration for this mission."},
      {"role": "assistant", "content": str(sample["configuration"])},
  ]

  text = tokenizer.apply_chat_template(messages, tokenize=False) # wraps text with special tokens depending on role (assitant or user)
  return encode_number(text)

dataset["text"] = dataset.apply(lambda x: generate_prompt(x, tokenizer),axis=1)

dataset = dataset.filter(["text"])
dataset.head()

,text
0,<s>[INST] The arena has a radius of <sn>1.2<mn...
1,"<s>[INST] The circular arena, constructed with..."
2,<s>[INST] The environment consists of a circul...
3,"<s>[INST] The circular arena, having a radius ..."
4,"<s>[INST] In this setting, a circular arena wi..."


In [9]:
print(dataset.iloc[0].text)

<s>[INST] The arena has a radius of <sn>1.2<mn>4.55<en> m. There are the following lights in the arena: ((<sn>1.2<mn>1.35<en>, <sn>1.2<mn>2.42<en>), (<sn>1.2<mn>0.73<en>, <sn>1.2<mn>1.01<en>), (-<sn>1.2<mn>2.66<en>, -<sn>1.2<mn>0.42<en>), (<sn>1.2<mn>1.61<en>, <sn>1.2<mn>2.49<en>)). Within a <sn>1.2<mn>2.23<en>-meter radius from the center, <sn>1<mn>5<en> robots are uniformly distributed. The mission's objective is for the robots to cover an area with a length of <sn>1.2<mn>5.52<en> meters and a width of <sn>1.2<mn>6.11<en> meters, all while staying connected. Two robots are considered connected if their distance is below <sn>1.2<mn>1.67<en> meters. 
Generate the xml configuration for this mission.[/INST] <?xml version="<sn>1.2<mn>1.00<en>" ?>
<config>
  <swarm-elems>
    <entity quantity="<sn>1<mn>5<en>" max_trials="<sn>3<mn>100<en>">
      <e-puck id="epuck">
        <controller config="automode_bt"/>
      </e-puck>
    </entity>
  </swarm-elems>
  <env-elems>
    <light id="light_<

convert dataset to huggingface dataset

In [10]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
generated_train_dataset, generated_val_dataset = train_test_split(dataset, test_size=0.2)

def to_dataset(df):
  dataset = ds.dataset(pa.Table.from_pandas(df).to_batches())

  ### convert to Huggingface dataset
  hg_dataset = Dataset(pa.Table.from_pandas(df))
  return hg_dataset

# def to_ppo_dataset(df):
#     hg_dataset = Dataset(pa.Table.from_pandas(df))

#generated_train_dataset = to_dataset(generated_train_dataset.head(250))
#generated_val_dataset = to_dataset(generated_val_dataset.head(50))
generated_train_dataset = to_dataset(generated_train_dataset.head(2500))
generated_val_dataset = to_dataset(generated_val_dataset.head(500))
generated_train_dataset

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 2000
})

In [11]:
def split_query_response(text,query_str="[/INST] "):
    
    spl = text.split(query_str)
    query = spl[0]+query_str
    resp = spl[1]
    return {"query": query, "response": resp}
    
dataset_ppo_train = generated_train_dataset.map(lambda x: split_query_response(x["text"]))
dataset_ppo_val = generated_train_dataset.map(lambda x: split_query_response(x["text"]))
dataset_ppo_train[0]["query"], dataset_ppo_train[0]["response"]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

('<s>[INST] In this setting, a rectangle is formed with dimensions <sn>1.2<mn>3.90<en> x <sn>1.2<mn>1.96<en> x <sn>1.2<mn>1.72<en>.There are <sn>1<mn>0<en> lights distributed evenly in the arena. Evenly positioned around the origin are <sn>2<mn>23<en> robots within a radius of <sn>1.2<mn>0.80<en> meters. The objective is for the robots to aggregate at the white circle. There are two designated areas on the floor: a circle at [<sn>1.2<mn>0.89<en>, <sn>1.2<mn>0.68<en>] in white, and another circle at [-<sn>1.2<mn>0.38<en>, -<sn>1.2<mn>0.94<en>] in black. \nGenerate the xml configuration for this mission.[/INST] ',
 '<?xml version="<sn>1.2<mn>1.00<en>" ?>\n<config>\n  <swarm-elems>\n    <entity quantity="<sn>2<mn>23<en>" max_trials="<sn>3<mn>100<en>">\n      <e-puck id="epuck">\n        <controller config="automode_bt"/>\n      </e-puck>\n    </entity>\n  </swarm-elems>\n  <env-elems>\n    <box id="wall_<sn>1<mn>0<en>" size="<sn>1.2<mn>0.01<en>,<sn>1.2<mn>1.96<en>,<sn>1.2<mn>0.10<en>" mov

In [12]:
generated_train_dataset[0]["text"]

'<s>[INST] In this setting, a rectangle is formed with dimensions <sn>1.2<mn>3.90<en> x <sn>1.2<mn>1.96<en> x <sn>1.2<mn>1.72<en>.There are <sn>1<mn>0<en> lights distributed evenly in the arena. Evenly positioned around the origin are <sn>2<mn>23<en> robots within a radius of <sn>1.2<mn>0.80<en> meters. The objective is for the robots to aggregate at the white circle. There are two designated areas on the floor: a circle at [<sn>1.2<mn>0.89<en>, <sn>1.2<mn>0.68<en>] in white, and another circle at [-<sn>1.2<mn>0.38<en>, -<sn>1.2<mn>0.94<en>] in black. \nGenerate the xml configuration for this mission.[/INST] <?xml version="<sn>1.2<mn>1.00<en>" ?>\n<config>\n  <swarm-elems>\n    <entity quantity="<sn>2<mn>23<en>" max_trials="<sn>3<mn>100<en>">\n      <e-puck id="epuck">\n        <controller config="automode_bt"/>\n      </e-puck>\n    </entity>\n  </swarm-elems>\n  <env-elems>\n    <box id="wall_<sn>1<mn>0<en>" size="<sn>1.2<mn>0.01<en>,<sn>1.2<mn>1.96<en>,<sn>1.2<mn>0.10<en>" movable="

## sft finetuning
necessary to have data be in distribution for ppo training according to their docs

In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments


# sft_arguments = TrainingArguments(
#     output_dir="logs",
#     num_train_epochs=4,
#     per_device_train_batch_size=1,
#     gradient_accumulation_steps=8, # 4
#     optim="paged_adamw_32bit",
#     save_steps=0,
#     logging_steps=25,
#     learning_rate=2e-4,
#     weight_decay=0.001,
#     #fp16=True,
#     bf16=False,
#     max_grad_norm=0.3,
#     max_steps=-1,
#     warmup_ratio=0.03,
#     group_by_length=True,
#     lr_scheduler_type="cosine",
#     report_to="none",
#     evaluation_strategy="epoch"
# )

# sft_trainer = SFTTrainer(
#     model=model,
#     train_dataset=generated_train_dataset,
#     eval_dataset=generated_val_dataset,
#     dataset_text_field="text",
#     tokenizer=tokenizer,
#     args=sft_arguments,
#     packing=False,
#     max_seq_length=2056,
# )

In [14]:
#sft_trainer.train()

In [15]:
#model.save_pretrained("sft_trained", from_pt=True)

In [16]:
_meval = AutoModelForCausalLM.from_pretrained("/kaggle/input/ppo-train-sft-24-08-27/sft_trained", quantization_config=bnb_config)
_meval = get_peft_model(_meval, lora_config)
text = _meval.generate(tokenizer.encode(generated_val_dataset[0]["text"], return_tensors="pt").to(_meval.device),**{
        "min_length": -1,
        # "top_k": 0.0,
        "top_p": 1.0,
        "do_sample": False,
        "pad_token_id": tokenizer.eos_token_id,
        "max_new_tokens": 2056,
        "stop_strings": ["</s>"],
        "tokenizer": tokenizer
    })

text = tokenizer.decode(text[0])
decode_number(text)

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

...

model-00006-of-00006.safetensors:   0%|          | 0.00/4.26G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'<s><s>[INST] A rectangular area, with a length of 3.21 meters, width of 6.95 meters, and height of 2.56 meters, is established.There are 3 lights distributed evenly in the arena. Evenly positioned around the origin are 17 robots within a radius of 1.31 meters. Present in the space are two circles—one situated at [-0.60, 0.10] with a radius of 0.35 meters, adorned in black, and another at [-0.84, 0.33] with a radius of 0.34 meters in white. The robots are tasked with transporting items from the white origing to the black circle. \nGenerate the xml configuration for this mission.[/INST] <?xml version="1.00" ?>\n<config>\n  <swarm-elems>\n    <entity quantity="17" max_trials="100">\n      <e-puck id="epuck">\n        <controller config="automode_bt"/>\n      </e-puck>\n    </entity>\n  </swarm-elems>\n  <env-elems>\n    <light id="light_0" position="1.12,-0.03,0.00" orientation="360,0,0" color="yellow" intensity="3.85" medium="leds"/>\n    <light id="light_1" position="-0.07,1.23,0.00" o

## ppo training

In [17]:
%pip install jellyfish

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.0/336.0 kB 7.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [18]:
model_vh = AutoModelForCausalLMWithValueHead.from_pretrained(_meval)

In [19]:
del _meval

In [20]:
import gc
#del sft_trainer
gc.collect()
torch.cuda.empty_cache()

In [21]:
import torch
from torch.nn import CrossEntropyLoss
import re
from jellyfish import jaro_similarity, levenshtein_distance
import re
import numpy as np
import ast
import sys
import os


# 2. initialize trainer
ppo_config = {"mini_batch_size": 1,"batch_size": 1, "optimize_cuda_cache": True, "is_peft_model": True, "learning_rate": 0.000000001}
config = PPOConfig(**ppo_config)
ppo_trainer = PPOTrainer(config=config, model=model_vh, tokenizer=tokenizer)
rewards = []
i = 0
for sample in range(len(dataset_ppo_train[:2])):
    i += 1

    # 3. encode a query
    #query_txt = "This morning I went to the "
    query_txt = dataset_ppo_train[0]["query"]
    label_txt = dataset_ppo_train[0]["response"]
    query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(model_vh.pretrained_model.device)
    label_tensor = tokenizer.encode(label_txt, return_tensors="pt").to(model_vh.pretrained_model.device)

    # 4. generate model response
    generation_kwargs = {
        "min_length": -1,
         "top_k": 0.0,
        "top_p": 1.0,
        "do_sample": True,
        #"pad_token_id": tokenizer.eos_token_id,
        "pad_token_id": tokenizer.unk_token_id,
       # "max_new_tokens": 1250,
        "max_new_tokens": 8,
        "stop_strings": ["</s>"],
        "tokenizer": tokenizer,
       # "kl_penalty": "full"
    }
    response_tensor = ppo_trainer.generate([item for item in query_tensor], return_prompt=False, **generation_kwargs)
    response_txt = tokenizer.decode(response_tensor[0])


    # 5. Define a reward function using the original loss
    def reward_fkt(response_tensor, label_tensor):
        try:
            # Print tensor shapes for debugging
            #print(f"response_tensor shape: {response_tensor.shape}")
            #print(f"label_tensor shape: {label_tensor.shape}")

            # Check if tensors are one-dimensional
            if len(response_tensor.shape) != 1:
                raise ValueError("response_tensor should be a 1-dimensional tensor")
            if len(label_tensor.shape) != 1:
                raise ValueError("label_tensor should be a 1-dimensional tensor")

            # Create dummy dimensions for demonstration
            num_classes = response_tensor.shape[0]  # Assuming this represents the number of classes

            # Adjusting example handling for 1D tensors
            # Assuming response_tensor is logits and label_tensor are labels directly
            shift_logits = response_tensor[:-1]  # Example shift, adjust if necessary
            shift_labels = label_tensor[1:]  # Example shift, adjust if necessary

            # Print shapes after shifting for debugging
            print(f"shift_logits shape: {shift_logits.shape}")
            print(f"shift_labels shape: {shift_labels.shape}")

            # Ensure tensors are the right shape
            if shift_labels.max().item() >= num_classes:
                raise ValueError("Labels contain values outside the range of the number of classes")

            # Reshape shift_logits to match the number of classes
            # Note: In a real scenario, you should ensure proper logits shape, this is just a placeholder
            shift_logits = shift_logits.view(-1, num_classes)
            shift_labels = shift_labels.view(-1)

            # Calculate cross-entropy loss
            loss_fct = CrossEntropyLoss(reduction='mean')
            loss = loss_fct(shift_logits, shift_labels)

            # Reward is the negative loss (lower loss = higher reward)
            reward = -loss.item()

            # Normalize reward (optional)
            return reward
        except Exception as e:
            print(f"Error calculating reward: {e}")
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)
            return 0.0

    reward = [torch.tensor(reward_fkt(response_tensor[0], label_tensor[0]), device=model_vh.pretrained_model.device)]

    # 6. train model with ppo
    train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)
    if i % 50 == 0:
        #ppo_trainer.log_stats(train_stats, {"query": "q", "response": "r"}, reward)
        reward = reward[0].cpu().numpy().item()
        print(reward)
        print(response_txt)
        rewards.append(reward)
    #train_stats

/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:267: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


shift_logits shape: torch.Size([7])
shift_labels shape: torch.Size([1148])
Error calculating reward: Labels contain values outside the range of the number of classes
<class 'ValueError'> 3545230856.py 73


/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1301: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1808.)
  std_scores = data["scores"].std()
/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1328: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1808.)
  stats["tokens/queries_len_std"] = torch.std(query_lens).cpu().numpy().item()
/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:1331: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1808.)
  stats["tok

shift_logits shape: torch.Size([7])
shift_labels shape: torch.Size([1148])
Error calculating reward: Labels contain values outside the range of the number of classes
<class 'ValueError'> 3545230856.py 73
shift_logits shape: torch.Size([7])
shift_labels shape: torch.Size([1148])
Error calculating reward: Labels contain values outside the range of the number of classes
<class 'ValueError'> 3545230856.py 73
shift_logits shape: torch.Size([7])
shift_labels shape: torch.Size([1148])
Error calculating reward: Labels contain values outside the range of the number of classes
<class 'ValueError'> 3545230856.py 73


In [22]:
# import torch
# from torch.nn import CrossEntropyLoss
# import re
# import numpy as np
# import ast
# from jellyfish import levenshtein_distance

# # 2. Initialize PPO trainer
# ppo_config = {"mini_batch_size": 1, "batch_size": 1, "optimize_cuda_cache": True, "is_peft_model": True, "learning_rate": 0.000000001}
# config = PPOConfig(**ppo_config)
# ppo_trainer = PPOTrainer(config=config, model=model_vh, tokenizer=tokenizer)
# rewards = []
# i = 0

# for sample in range(len(dataset_ppo_train)):
#     i += 1

#     # 3. Encode a query
#     query_txt = dataset_ppo_train[0]["query"]
#     label_txt = dataset_ppo_train[0]["response"]
#     query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(model_vh.pretrained_model.device)
#     label_tensor = tokenizer.encode(label_txt, return_tensors="pt").to(model_vh.pretrained_model.device)

#     # 4. Generate model response
#     generation_kwargs = {
#         "min_length": -1,
#         "top_k": 0.0,
#         "top_p": 1.0,
#         "do_sample": True,
#         "pad_token_id": tokenizer.unk_token_id,
#         "max_new_tokens": 1050,
#         "stop_strings": ["</s>"],
#         "tokenizer": tokenizer,
#     }
#     response_tensor = ppo_trainer.generate([query_tensor], return_prompt=False, **generation_kwargs)
#     response_txt = tokenizer.decode(response_tensor[0])

#     # 5. Define a reward function using the original loss
#     def reward_fkt(model, query_tensor, response_tensor, label_tensor):
#         try:
#             # Shift logits and labels for computing cross-entropy loss
#             shift_logits = response_tensor[..., :-1, :].contiguous()
#             shift_labels = label_tensor[..., 1:].contiguous()

#             # Calculate cross-entropy loss
#             loss_fct = CrossEntropyLoss(reduction='mean')
#             loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

#             # Reward is the negative loss (lower loss = higher reward)
#             reward = -loss.item()

#             # Normalize reward (optional)
#             return reward
#         except Exception as e:
#             print(f"Error calculating reward: {e}")
#             return 0.0

#     # Calculate reward based on the model's loss
#     reward = [torch.tensor(reward_fkt(model_vh, query_tensor, response_tensor, label_tensor), device=model_vh.pretrained_model.device)]

#     # 6. Train model with PPO
#     train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)
#     if i % 1 == 0:
#         reward = reward[0].cpu().numpy().item()
#         print(reward)
#         print(response_txt)
#         rewards.append(reward)


In [23]:
response_txt

' Given the complexity of the problem, it'

In [24]:
rewards

[]

In [25]:
!mkdir -p ppo_trained

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
model_vh.save_pretrained("ppo_trained", from_pt=True)

In [27]:
# import gc
# del model
# del tokenizer
# gc.collect()
# torch.cuda.empty_cache()

In [28]:
del ppo_trainer
gc.collect()
torch.cuda.empty_cache()